<a href="https://colab.research.google.com/github/SruthiM-10/Irrigation-Scheduling-Recommendations-And-Dataset/blob/main/b/(5)_prep_for_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import re
import json
import requests
import os
import warnings
import joblib
from pathlib import Path
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_predict
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

In [ ]:
CSV_PATH = "/content/drive/MyDrive/1:1_Sruthi_Manoj/Dataset/csv_files/Extra analysis/cleaned_samples (8).csv"
OUTPUT_DIR = "/content/drive/MyDrive/1:1_Sruthi_Manoj/Dataset/processed_csv_files (prep for training models)"

In [ ]:
df = pd.read_csv(CSV_PATH)
df.drop("Unnamed: 0", axis= 1, inplace= True)
df

,irrigation_scheduling_method,irrigation_scheduling_method_standardized,plant_yield,plant_yield_kg_per_hectare,plant_yield_notes,location,latitude,latitude_decimal_degrees,latitude_notes,lat_filled,...,water_productivity_kg_per_m^3,water_productivity_notes,water_productivity_valid,water_productivity_filled,file_path,doi,doi_url,processed_at,content source,relevance_score
0,Soil moisture sensor-based method (T-VWC),Soil Moisture-Based,66.77 t ha−1,60572.74245,This value is given in tons per hectare.,Xiaotangshan National Precision Agriculture Ex...,40.1786° N,40.178600,Original value was in degrees with a direction...,False,...,17.370000,NaN,NaN,False,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,2025-09-12 01:42:16,pdf,9
1,Accumulated radiation combined with soil moist...,Soil Moisture-Based,58.98 t ha−1,53505.77130,Yield value is given in tons per hectare.,Xiaotangshan National Precision Agriculture Ex...,40.1786° N,40.178600,Original value was in degrees with a direction...,False,...,20.520000,NaN,NaN,False,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,2025-09-12 01:42:16,pdf,9
2,Crop evapotranspiration estimated method using...,Evapotranspiration-Based,64.54 t ha−1,58549.71990,Value is given in tons per hectare.,Xiaotangshan National Precision Agriculture Ex...,40.1786° N,40.178600,Original value was in degrees with a direction...,False,...,25.070000,NaN,NaN,False,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,2025-09-12 01:42:16,pdf,9
3,Regulated Deficit Irrigation,Deficit/Partial Irrigation,2.406 kg m-2,24060.00000,"Value is in kilograms per square meter, which ...","Maringá, Paraná, Brazil",23º25′S,-23.416667,Converted from degrees and minutes format (23º...,False,...,13.583452,NaN,NaN,True,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1093/jxb/erv034,https://doi.org/10.1093/jxb/erv034,2025-09-12 04:06:41,pdf,9
4,Regulated Deficit Irrigation,Deficit/Partial Irrigation,2.540 kg m-2,25400.00000,"Value is in kg per square meter, which can be ...","Maringá, Paraná, Brazil",23º25′S,-23.416667,Converted from degrees and minutes (23º25′S) t...,False,...,13.583452,NaN,NaN,True,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1093/jxb/erv034,https://doi.org/10.1093/jxb/erv034,2025-09-12 04:06:41,pdf,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290,"100% ETc, 4-day interval",Evapotranspiration-Based,"136.78 t/ha (Spring 2023), 50.10 t/ha (Autumn ...",45449.96850,Yield value for Autumn 2023.,"Yangling, Shaanxi, China",34°20′N,34.333333,Original value was in degrees and minutes form...,False,...,19.300000,Autumn 2023,True,False,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.3390/agronomy15051068,https://doi.org/10.3390/agronomy15051068,2025-09-12 17:23:02,pdf,10
291,Timing of last irrigation,Conventional/Fixed Scheduling,1669-2405 kg/ha,2405.00000,Value is already in kg/ha.,"Gezira Research Station Farm, Wad Medani, Sudan",NaN,14.401180,NaN,True,...,0.710000,Maximum value of the range 0.45-0.71 kg/m3,True,False,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.21608/sjdr.2020.366016074,https://doi.org/10.21608/sjdr.2020.366016074,2025-09-12 23:58:14,pdf,8
292,Film-Mulched Drip Irrigation,Conventional/Fixed Scheduling,"6218.07 kg ha−1 (2018), 5660.59 kg ha−1 (2019)",5660.59000,Yield value for the year 2019.,"Korla, Xinjiang, China",40°53′N,40.883333,Original value was in degrees and minutes form...,False,...,2.600000,(W 0.8),True,False,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.3390/agronomy15071674,https://doi.org/10.3390/agronomy15071674,2025-09-13 00:18:02,pdf,9
293,Film-Mulched Drip Irrigation,Conventional/Fixed S

In [ ]:
df.columns

Index(['irrigation_scheduling_method',
       'irrigation_scheduling_method_standardized', 'plant_yield',
       'plant_yield_kg_per_hectare', 'plant_yield_notes', 'location',
       'latitude', 'latitude_decimal_degrees', 'latitude_notes', 'lat_filled',
       'longitude', 'longitude_decimal_degrees', 'longitude_notes',
       'long_filled', 'altitude', 'altitude_numeric', 'altitude_notes',
       'altitude_filled', 'plant_type', 'plant_type_standardized',
       'plant_type_notes', 'water_productivity',
       'water_productivity_kg_per_m^3', 'water_productivity_notes',
       'water_productivity_valid', 'water_productivity_filled', 'file_path',
       'doi', 'doi_url', 'processed_at', 'content source', 'relevance_score'],
      dtype='object')

###Loading and filtering data
Load the CSV into a DataFrame

Choose the correct column for plant type

Keep only numeric columns (latitude/longitude) + plant + method

Remove rows with missing data

Clean up labels

In [ ]:
#CONSTANTS
TARGET_COL = "irrigation_scheduling_method_standardized"
CANDIDATE_PLANT_COLS  = ["plant_type_standardized"]
NUMERIC_GEO_COLS = ["latitude_decimal_degrees", "longitude_decimal_degrees", "altitude_numeric"]

In [ ]:
#Checks for plant type standardized or plant type column
def choose_plant_column(df: pd.DataFrame) -> str:
    for col in CANDIDATE_PLANT_COLS:
        if col in df.columns:
            return col
    raise ValueError(f"Need one of {CANDIDATE_PLANT_COLS} in the CSV.")

In [ ]:
def load_and_filter(csv_path: str):
    df = pd.read_csv(csv_path)

    plant_col = choose_plant_column(df)

    num_cols = [c for c in NUMERIC_GEO_COLS if c in df.columns]
    if len(num_cols) != 3:
        raise ValueError(f"Expected {NUMERIC_GEO_COLS} in the CSV.")

    keep_cols = [TARGET_COL, plant_col] + num_cols
    small = df[keep_cols].copy()

    # Drop missing/empty target or features
    small = small.dropna(subset=[TARGET_COL, plant_col] + num_cols)
    small = small[small[TARGET_COL].astype(str).str.strip() != ""]
    small = small[small[plant_col].astype(str).str.strip() != ""]

    # Cast lat/lon to numeric
    for c in num_cols:
        small[c] = pd.to_numeric(small[c], errors="coerce")
    small = small.dropna(subset=num_cols)

    # Trim spaces from target labels
    small[TARGET_COL] = small[TARGET_COL].astype(str).str.strip()

    small =small.reset_index(drop=True) #reset index
    return small, plant_col, num_cols

In [ ]:
df_filtered, plant_col, num_cols = load_and_filter(CSV_PATH)
df_filtered

,irrigation_scheduling_method_standardized,plant_type_standardized,latitude_decimal_degrees,longitude_decimal_degrees,altitude_numeric
0,Soil Moisture-Based,Cucumber,40.178600,116.444200,37.0
1,Soil Moisture-Based,Cucumber,40.178600,116.444200,37.0
2,Evapotranspiration-Based,Cucumber,40.178600,116.444200,37.0
3,Deficit/Partial Irrigation,Broccoli,-23.416667,-51.950000,542.0
4,Deficit/Partial Irrigation,Broccoli,-23.416667,-51.950000,542.0
...,...,...,...,...,...
290,Evapotranspiration-Based,Tomato,34.333333,108.066667,521.0
291,Conventional/Fixed Scheduling,Cotton,14.401180,33.519890,409.0
292,Conventional/Fixed Scheduling,Cotton,40.883333,86.933333,900.0
293,Conventional/Fixed Scheduling,Cotton,40.883333,86.933333,900.0


# Train - Test Split

In [ ]:
X = df_filtered[[plant_col] + num_cols]
y = df_filtered[TARGET_COL]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, stratify=y, random_state=42
)
X_train.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)

print(f"Train rows: {len(X_train)} | Test rows: {len(X_test)}")
print(f"Plant column: {plant_col} | Numeric columns: {num_cols}")
print(f"Unique categories: {y.nunique()}")

Train rows: 236 | Test rows: 59
Plant column: plant_type_standardized | Numeric columns: ['latitude_decimal_degrees', 'longitude_decimal_degrees', 'altitude_numeric']
Unique categories: 5


In [ ]:
ohe = OneHotEncoder(drop="first", sparse_output=False, handle_unknown="ignore")
X_train_cat = ohe.fit_transform(X_train[[plant_col]])
X_test_cat = ohe.transform(X_test[[plant_col]])
ohe_labels = ohe.get_feature_names_out([plant_col])
X_train_cat = pd.DataFrame(X_train_cat, columns=ohe_labels, index=X_train.index)
X_test_cat = pd.DataFrame(X_test_cat, columns=ohe_labels, index=X_test.index)

In [ ]:
# Save encoder
import pickle
ohe_path = os.path.join(OUTPUT_DIR, "onehot_encoder.pkl")
with open(ohe_path, "wb") as f:
    pickle.dump(ohe, f)
print(f"OneHotEncoder saved at: {ohe_path}")

OneHotEncoder saved at: /content/drive/MyDrive/1:1_Sruthi_Manoj/Dataset/processed_csv_files (prep for training models)/onehot_encoder.pkl


In [ ]:
scaler = StandardScaler()
X_train_num = pd.DataFrame(scaler.fit_transform(X_train[num_cols]), columns=num_cols, index=X_train.index)
X_test_num = pd.DataFrame(scaler.transform(X_test[num_cols]), columns=num_cols, index=X_test.index)

In [ ]:
with open(os.path.join(OUTPUT_DIR, "scaler.pkl"), "wb") as f:
    joblib.dump(scaler, f)

In [ ]:
#reset index
X_train_cat.reset_index(drop=True, inplace=True)
X_test_cat.reset_index(drop=True, inplace=True)
X_train_num.reset_index(drop=True, inplace=True)
X_test_num.reset_index(drop=True, inplace=True)

In [ ]:
# Combine Encoded + Scaled + Target
train_final = pd.concat([X_train_cat, X_train_num, y_train], axis=1)
test_final = pd.concat([X_test_cat, X_test_num, y_test], axis=1)

In [ ]:
train_final.reset_index(drop=True, inplace=True)
test_final.reset_index(drop=True, inplace=True)

In [ ]:
# Save Final CSVs
train_csv = os.path.join(OUTPUT_DIR, "train_preprocessed.csv")
test_csv = os.path.join(OUTPUT_DIR, "test_preprocessed.csv")
train_final.to_csv(train_csv, index=False)
test_final.to_csv(test_csv, index=False)
print(f"Train saved to: {train_csv}")
print(f"Test saved to: {test_csv}")

train_final.head()

Train saved to: /content/drive/MyDrive/1:1_Sruthi_Manoj/Dataset/processed_csv_files (prep for training models)/train_preprocessed.csv
Test saved to: /content/drive/MyDrive/1:1_Sruthi_Manoj/Dataset/processed_csv_files (prep for training models)/test_preprocessed.csv


,plant_type_standardized_Barley,plant_type_standardized_Bell pepper,plant_type_standardized_Broccoli,plant_type_standardized_Cabbage,plant_type_standardized_Cotton,plant_type_standardized_Cucumber,plant_type_standardized_Grapes,plant_type_standardized_Green Pea,plant_type_standardized_Groundnut,plant_type_standardized_Haricot bean,...,plant_type_standardized_Sunflower,plant_type_standardized_Sweet potato,plant_type_standardized_Tea,plant_type_standardized_Tobacco,plant_type_standardized_Tomato,plant_type_standardized_Wheat,latitude_decimal_degrees,longitude_decimal_degrees,altitude_numeric,irrigation_scheduling_method_standardized
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.507646,-0.136336,-1.223580,Deficit/Partial Irrigation
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.671144,1.246195,1.705966,Conventional/Fixed Scheduling
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.835372,-0.748297,-0.025663,Conventional/Fixed Scheduling
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,-1.657556,-0.138834,0.940589,Conventional/Fixed Scheduling
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,-1.657556,-0.138834,0.940589,Conventional/Fixed Scheduling


In [ ]:
test_final.head()

,plant_type_standardized_Barley,plant_type_standardized_Bell pepper,plant_type_standardized_Broccoli,plant_type_standardized_Cabbage,plant_type_standardized_Cotton,plant_type_standardized_Cucumber,plant_type_standardized_Grapes,plant_type_standardized_Green Pea,plant_type_standardized_Groundnut,plant_type_standardized_Haricot bean,...,plant_type_standardized_Sunflower,plant_type_standardized_Sweet potato,plant_type_standardized_Tea,plant_type_standardized_Tobacco,plant_type_standardized_Tomato,plant_type_standardized_Wheat,latitude_decimal_degrees,longitude_decimal_degrees,altitude_numeric,irrigation_scheduling_method_standardized
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.464342,-0.139521,-1.201733,Conventional/Fixed Scheduling
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.835372,-0.748297,-0.025663,Conventional/Fixed Scheduling
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.931113,0.907744,0.091636,Conventional/Fixed Scheduling
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.671144,1.246195,1.705966,Conventional/Fixed Scheduling
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.835372,-0.748297,-0.025663,Conventional/Fixed Scheduling


In [ ]:
train_final.isna().sum()

,0
plant_type_standardized_Barley,0
plant_type_standardized_Bell pepper,0
plant_type_standardized_Broccoli,0
plant_type_standardized_Cabbage,0
plant_type_standardized_Cotton,0
plant_type_standardized_Cucumber,0
plant_type_standardized_Grapes,0
plant_type_standardized_Green Pea,0
plant_type_standardized_Groundnut,0
plant_type_standardized_Haricot bean,0
